In [1]:
from PIL import Image, ImageDraw, ImageOps
import numpy as np

# ==============================================================================
radius = 120
resize = 80

padding = 50
center0 = (163,179)
step_width = 266
step_height = 348
# ==============================================================================

# Load the original image
img_path = 'shibainus.webp'
original_img = Image.open(img_path)

# 上下左右に余白を300px追加
original_img = ImageOps.expand(original_img, padding, fill='white')

# Calculate the size of each individual image (assuming a 4x4 grid)
width, height = original_img.size


# Function to create circular mask
def create_circular_mask(radius):
    h = radius*2
    w = radius*2
    center = (int(radius), int(radius))

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = (dist_from_center <= radius)
    return mask


for xi in range(4):
    for yi in range(3):

        center_x = center0[0] + xi*step_width
        center_y = center0[1] + yi*step_height

        left = center_x - radius
        top  = center_y - radius
        right = left + radius*2
        bottom = top + radius*2

        # Crop the image to the current grid cell
        crop_img = original_img.crop((left, top, right, bottom))

        # Create a circular mask
        mask = create_circular_mask(radius)
        mask_img = Image.fromarray((mask * 255).astype(np.uint8))

        # Apply the mask to the cropped image
        circular_crop_img = ImageOps.fit(crop_img, mask_img.size, centering=(0.5, 0.5))
        circular_crop_img.putalpha(mask_img)
        
        # 画像を80x80にリサイズ
        circular_crop_img = circular_crop_img.resize((resize,resize))
        
        circular_crop_img.save(f'shibainu_{xi+1}_{yi+1}.png')